In [6]:
import pandas as pd 
import numpy as np 

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

options = Options()
options.add_argument('--blink-settings=imagesEnabled=false')
#options.add_argument('--enable-print-browser')
options.add_argument("--headless")
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "eager"

import time
import os

from bs4 import BeautifulSoup
import requests

headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
import urllib.request

tickers = list(pd.read_excel('لیست صندوق_های سهامی.xlsx')['نماد'].values)
tickers.sort()

def date_gen(item):
    text = item.text
    loc = text.find('/')
    date = int(text[loc - 4 : loc + 6].replace('/', ''))
    eslahie = 'اصلاحیه' in text
    return date, eslahie

def date_check(date, sent):
    date_m = (((date // 100) % 100) + 1) % 12
    sent_m = ((sent // 100) % 100) % 12
    date_y = date // 10000 if date_m != 1 else (date // 10000) + 1
    sent_y = sent // 10000
    return (date_m == sent_m) and (date_y == sent_y)


In [11]:
link_start = 'https://codal.ir'
start_url = "https://codal.ir/ReportList.aspx?search&Symbol="
end_url = "&LetterType=-1&FromDate=1398%2F01%2F01&Isic=46430170&AuditorRef=-1&Audited&NotAudited&IsNotAudited=false&Childs&Mains&Publisher=false&CompanyState=2&Category=3&CompanyType=-1&Consolidatable&NotConsolidatable&PageNumber="
driver = webdriver.Chrome(options = options, desired_capabilities = caps)
for ticker in tickers[75:]:
    os.mkdir(os.getcwd() + '/New_Data/' + ticker)
    page = '1'
    url = start_url + ticker + end_url + page
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'letter-title')))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        page_num = int(soup.find_all('li', {'title' : 'آخرین صفحه'})[0].find('a')['href'].split('=')[-1])
    except:
        page_num = 1
    a = []
    sent = []
    a += soup.find_all('a', class_ = 'letter-title')
    sent += soup.find_all('td', {'data-heading' : 'زمان ارسال'})
    if page_num != 1:
        for page in range(2, page_num+1):
            url = start_url + ticker + end_url + str(page)
            driver.get(url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'letter-title')))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            a += soup.find_all('a', class_ = 'letter-title')
            sent += soup.find_all('td', {'data-heading' : 'زمان ارسال'})
    dates = [date_gen(x)[0] for x in a]
    eslahie = [date_gen(x)[1] for x in a]
    sent_dates = [date_gen(td.findChild())[0] for td in sent]
    links = [link_start + x['href'] for x in a]
    j = 0
    for i, link in enumerate(links):
        r = requests.get(link, headers = headers, allow_redirects = False)
        soup2 = BeautifulSoup(r.text, 'html.parser')
        imgs = soup2.find_all('img')
        for img in imgs:
            if 'xls' in img['src']:
                download_link = 'https://codal.ir/Reports/' + img.parent.parent['onclick'].split("'")[1]
                excel_file = requests.get(download_link, headers = headers, allow_redirects = True)
                if (date_check(dates[i], sent_dates[i])) and not(eslahie[i]):
                    open(os.getcwd() + '/New_Data/' + ticker + '/' + str(dates[i]) + '.xlsx', 'wb').write(excel_file.content)
                else:
                    open(os.getcwd() + '/New_Data/' + ticker + '/' + str(dates[i]) + '_check' + str(j) + '.xlsx', 'wb').write(excel_file.content)
                    j += 1
                break
    